In [1]:
import numpy as np
import pandas as pd
import cv2
import os
from os import listdir
from os.path import join
import matplotlib.pyplot as plt

# Loading Dataset

In [2]:
index_dir={'0':0,'1':1,'2':2,'3':3,'4':4,'5':5,'6':6,'7':7,'8':8,'9':9,'+':10,'-':11,'times':12}

In [3]:
def get_index(directory):
    return index_dir[directory]

def load_images(folder):
    train_data = []

    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename), cv2.IMREAD_GRAYSCALE) 
        #img = ~img
        if img is not None:
            img = ~img
            _, img = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)
            ctrs, _ = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
            cnt = sorted(ctrs, key=lambda ctr: cv2.boundingRect(ctr)[0]) 
            m = 0
            for c in cnt:
                x, y, w, h = cv2.boundingRect(c)
                m = max(w*h, m)
                if m == w*h:
                    x_max,y_max,w_max,h_max=x,y,w,h
            im_crop = img[y_max:y_max+h_max+10, x_max:x_max+w_max+10] 
            im_resize = cv2.resize(im_crop, (28, 28)) 
            im_resize = np.reshape(im_resize, (784, 1)) 
            train_data.append(im_resize)
    return train_data

In [4]:

dataset_dir = './dataset/'

# Only keep folders (ignore files like .npy)
directory_list = [d for d in os.listdir(dataset_dir) if os.path.isdir(os.path.join(dataset_dir, d))]

first = True
data = []

print("Importing...")

for directory in directory_list:
    print(directory)
    
    if first:
        first = False
        data_1 = load_images(dataset_dir + directory)
        for i in range(0, len(data_1)):
            data.append(np.append(data_1[i], [str(get_index(directory))]))
        continue

    auxiliary_data = load_images(dataset_dir + directory)
    for i in range(0, len(auxiliary_data)):
        data.append(np.append(auxiliary_data[i], [str(get_index(directory))]))

# Convert to dataframe and save as CSV
df = pd.DataFrame(data, index=None)
df.to_csv('model/train_data.csv', index=False)


Importing...
CompleteImages
CROHME_test_2011
CROHME_training_2011
dataset
MatricesTest2014
MatricesTrain2014
testData_2012
TestINKML_2013
trainData_2012_part1
trainData_2012_part2
TrainINKML_2013
